In [202]:
from glob import glob
import urllib
import earthaccess
from datatree import open_datatree
from PIL import Image
import xarray as xr
from satpy.scene import Scene
import os
import re
from pyresample import create_area_def
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


In [203]:
def download_files(level, bounding_box, temporal_limits):
    '''Download data from EarthData (NOT cloud access)'''
    
    earthaccess.login()
    #password is OHW2023ohw2023!
    
    if level == 'L1b':
        short_names = ["VNP02MOD", "VNP02IMG", "VNP02DNB", "VNP03MOD", "VNP03IMG", "VNP03DNB"]#,
                      #"VJ02MOD", "VJ02IMG", "VJ02DNB", "VJ03MOD", "VJ03IMG", "VJ03DNB"]
    elif level == 'L2':
        short_names = ["VNP29"]
    
    for short_name in short_names:
        results = earthaccess.search_data(
        short_name=short_name, 
        cloud_hosted=True, 
        bounding_box=bounding_box,
        temporal=temporal_limits
        )
        
        earthaccess.download(results, './data/' + level + '/') #Separate directories for each product
        #earthaccess.download(results, './data/' + level + '/' + short_names + '/') #Separate directories for each product

In [204]:
def setup_projection(bounding_box):
    '''Set up a custom area and projection for the images'''
    
    # Bounding box coordinates
    area_id = 'skq2022'
    description = 'Beaufort Sea 2022 Stereographic Projection'

    area_extent = (bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3])

    width = 1100  # Specify the desired width in pixels
    height = 550  # Specify the desired height in pixels

    projection = {
        'proj': 'stere',
        'lat_0': 90.0,       # Latitude of the projection's center
        'lon_0': -145.0,     # Longitude of the projection's center
        'lat_ts': 60.0,      # Latitude where true scale is used
        'a': 6378273,        # Semi-major axis of the ellipsoid (WGS84)
        'b': 6356889.449,    # Semi-minor axis of the ellipsoid (WGS84)
    }

    #create_area_def?
    my_area = create_area_def(area_id, description = description, projection = projection,
                              width=width, height=height,
                              area_extent=area_extent, units='degrees')

    print(my_area)

    lons, lats = my_area.get_lonlats()

    print("Longitude Range:", lons.min(), lons.max()) 
    print("Latitude Range:", lats.min(), lats.max())

    print("Pixel size: x = ", my_area.pixel_size_x, "m, y = ", my_area.pixel_size_y, "m") #horizontal resolution in meters
    
    return(my_area)


In [205]:
def get_day_night(filenames):
    all_flags = [None] * len(filenames)
    for i, file in enumerate(filenames):
        dt = open_datatree(file)
        all_flags[i] = dt.attrs['DayNightFlag']
        
    # Check if all strings are the same
    all_same = all(flag == all_flags[0] for flag in all_flags)
    if all_same:
        return(all_flags[0])
    
    else:
        #print('Unclear when image is taken. Files are:', all_flags)
        
        if 'Day' in all_flags and 'Night' not in all_flags: #Some combo of 'Day' and 'Both'
            return('Day')
        elif 'Night' in all_flags and 'Day' not in all_flags: #Some combo of 'Night' and 'Both'
            return('Night')
        elif 'Day' in all_flags and 'Night' in all_flags: #Some combo of 'Day' and 'Night' - will make both images
            return('Both')
        else:
            return(None)  # No valid combination found


In [206]:
def get_file_times(filenames):
    '''Generate a list of unique, sorted day and time strings for all files.'''
    
    pattern = r'\.A(\d{7}).*(\d{4})\.001\.'  # Capture both day and time components of the filename
    time_strings = [None] * len(filenames)  # Preallocate with None
    for i, filename in enumerate(filenames):
        match = re.search(pattern, filename)
        if match: 
            day = match.group(1)
            hour = match.group(2)
            time_strings[i] = day + '.' + hour
    
    unique_sorted_time_strings = sorted(set(time_strings))
    return(unique_sorted_time_strings) 

In [234]:
def get_filenames_at_current_time(i, filetimes):
    '''Generate a list of all filenames at the current timestep. If two sets of files
        are taken six minutes apart, assume they are part of the same swath and group them together.'''
    
    filetime = filetimes[i]
    filenames = glob(data_path+'L1b/VNP*'+filetime+'.001*nc') #Files at this timestep

    #Check if the next batch of files is within six minutes and should be combined
    if i < len(filetimes)-1:
        nextfiletime = filetimes[i+1]
        
        # Convert timestamp strings to datetime objects
        filetime_dt = datetime.strptime(filetime, "%Y%j.%H%M")
        nextfiletime_dt = datetime.strptime(nextfiletime, "%Y%j.%H%M")

        # Calculate time difference
        time_difference = nextfiletime_dt - filetime_dt
        combine_files = time_difference <= timedelta(minutes=6)
        
        if combine_files: 
            print("Combining files at " + filetime[-4:] + ' and ' + nextfiletime[-4:])
            nextfilenames = glob(data_path+'L1b/VNP*'+nextfiletime+'.001*nc')  
            filenames.extend(nextfilenames)
    
    #Check if this batch of files was already combined with the previous batch of files
    if i > 0:
        prevfiletime = filetimes[i-1]
        prevfiletime_dt = datetime.strptime(prevfiletime, "%Y%j.%H%M")

        # Calculate time difference
        time_difference = filetime_dt - prevfiletime_dt
        already_combined = time_difference <= timedelta(minutes=6)
        if already_combined:
            print("Files at " + filetime[-4:] + ' were aleady combined with ' + prevfiletime[-4:]+ '. Will not reprocess the files')
            filenames = None
    
    #print(filenames)
    return filenames

In [208]:
def choose_composite(filenames):
    '''Decide which composite to generate for this batch of files''' 
    day_night_flag = get_day_night(filenames)
    print('Image is ', day_night_flag)

    if day_night_flag == 'Day':
        composite = 'true_color'
    elif day_night_flag == 'Night' or day_night_flag == 'Both':
        composite = 'adaptive_dnb'
    elif day_night_flag == 'Both':
        print('Not making image because day_night_flag is Both')
        composite = None
        #Need to decide what to do...maybe make both composites?
    return composite
   

In [254]:
#Set up time and geographic range, download files

bounding_box=(-160.0, 73.0, -130.0, 77.0)
temporal_limits=("2022-10-07", "2022-10-10")
#download_files(level='L1b', bounding_box=bounding_box, temporal_limits=temporal_limits)
#download_files(level='L2', bounding_box=bounding_box, temporal_limits=temporal_limits)


In [217]:
#Create a list of times at which images were taken. These are directly from the file names
#The format is YYYYDDD.HHMM

data_path = '/Users/lcrews/Documents/Python/sea_ice_oscillations/data/'
all_filenames = glob(data_path+'L1b/VNP*.001*nc')
filetimes = get_file_times(all_filenames)
print(filetimes)

['2022280.0124', '2022280.0130', '2022280.0818', '2022280.1000', '2022280.1006', '2022280.1142', '2022280.1324', '2022280.1506', '2022280.1642', '2022280.1648', '2022280.1824', '2022280.2006', '2022280.2142', '2022280.2148', '2022280.2324', '2022280.2330', '2022281.0106', '2022281.0942', '2022281.1124', '2022281.1306', '2022281.1448', '2022281.1624', '2022281.1630', '2022281.1806', '2022281.1942', '2022281.1948', '2022281.2124', '2022281.2130', '2022281.2306', '2022282.0048', '2022282.0924', '2022282.1106', '2022282.1248', '2022282.1424', '2022282.1430', '2022282.1606', '2022282.1748', '2022282.1924', '2022282.1930', '2022282.2106', '2022282.2248']


In [235]:
#Create reprojected images at each timestep

my_area = setup_projection(bounding_box) #Setup projection      

#Make  images for each satellite pass
for i, filetime in enumerate(filetimes):
    print() # This will print a blank line to distinguish this time's information
    
    #All files at the current time - combine files on same swath (six minutes apart)
    filenames = get_filenames_at_current_time(i, filetimes)
    if filenames is None: #This happens if the timestep was part of the same swath and already plotted at previous timestep
        continue  
    
    scn = Scene(filenames=filenames, reader='viirs_l1b')
        
    #channels = choose_composite(filenames)
    channels = ['true_color', 'adaptive_dnb']#, 'I02', 'M09'] #Trying I02 based on https://twitter.com/bill_line/status/1690022728441716736
    
    #Remove any datasets composites that are not available for this scene
    available = scn.available_composite_names() + scn.available_dataset_names()
    channels = list(filter(lambda item: item in available, channels))
        
    for channel in channels:
        
        savename = './images/VNP_%s_%s.png' % (filetime, channel)
        if os.path.exists(savename): #Dont remake this figure
            continue

        try:
            #Generate composite and resample to the common grid my_area defined in setup_projection
            scn.load([channel], generate=False)
            resampled_scene = scn.resample(my_area)
        
            print('Generating ' + channel + ' at ' + filetime)

            #Create numpy arrays to use for image analysis
            scn_to_numpy(resampled_scene, channel)

            #Write to an image .png
            resampled_scene.save_dataset(channel, filename = savename)
        
        except Exception as e:
            print(f"An error occurred while processing '{channel}' for '{filetime}': {e}")
    

Area ID: skq2022
Description: Beaufort Sea 2022 Stereographic Projection
Projection: {'a': '6378273', 'lat_0': '90', 'lat_ts': '60', 'lon_0': '-145', 'no_defs': 'None', 'proj': 'stere', 'rf': '298.279411123064', 'type': 'crs', 'units': 'm', 'x_0': '0', 'y_0': '0'}
Number of columns: 1100
Number of rows: 550
Area extent: (-461819.8016, -1723534.9635, 352116.2306, -1314115.6629)
Longitude Range: -164.34357984307965 -130.01910355902044
Latitude Range: 73.0042764882928 77.43597501958925
Pixel size: x =  739.9418474860364 m, y =  744.3987284632565 m

Combining files at 0124 and 0130
Generating M09 at 2022280.0124


/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)



Files at 0130 were aleady combined with 0124. Will not reprocess the files


Combining files at 1000 and 1006


Failed to load DataID(name='M03', wavelength=WavelengthRange(min=0.478, central=0.488, max=0.498, unit='µm'), resolution=742, calibration=<1>, modifiers=()) from <VIIRSL1BFileHandler: '/Users/lcrews/Documents/Python/sea_ice_oscillations/data/L1b/VNP02MOD.A2022280.1006.001.2022280180434.nc'>
Traceback (most recent call last):
  File "/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/satpy/readers/yaml_reader.py", line 699, in _load_dataset
    projectable = fh.get_dataset(dsid, ds_info)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/satpy/readers/viirs_l1b.py", line 203, in get_dataset
    metadata = self.get_metadata(dataset_id, ds_info)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/satpy/readers/viirs_l1b.py", line 182, in get_metadata
    i = getattr(self[var_path], 'attrs

An error occurred while processing 'true_color': Chunks do not add up to shape. Got chunks=((4096, 2368), (4096, 2304)), shape=(12960, 6400)
An error occurred while processing 'adaptive_dnb': Chunks do not add up to shape. Got chunks=((4096, 2368), (4096, 2304)), shape=(12960, 6400)


...

Failed to load DataID(name='M09', wavelength=WavelengthRange(min=1.371, central=1.378, max=1.386, unit='µm'), resolution=742, calibration=<1>, modifiers=()) from <VIIRSL1BFileHandler: '/Users/lcrews/Documents/Python/sea_ice_oscillations/data/L1b/VNP02MOD.A2022282.1430.001.2022282204700.nc'>
Traceback (most recent call last):
  File "/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/satpy/readers/yaml_reader.py", line 699, in _load_dataset
    projectable = fh.get_dataset(dsid, ds_info)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/satpy/readers/viirs_l1b.py", line 203, in get_dataset
    metadata = self.get_metadata(dataset_id, ds_info)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/satpy/readers/viirs_l1b.py", line 182, in get_metadata
    i = getattr(self[var_path], 'attrs

An error occurred while processing 'M09': Chunks do not add up to shape. Got chunks=((4096, 2368), (4096, 2304)), shape=(12960, 6400)

Files at 1430 were aleady combined with 1424. Will not reprocess the files

Generating M09 at 2022282.1606


/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)



Generating M09 at 2022282.1748

Combining files at 1924 and 1930
Generating M09 at 2022282.1924

Files at 1930 were aleady combined with 1924. Will not reprocess the files

Generating M09 at 2022282.2106



UnboundLocalError: cannot access local variable 'filetime_dt' where it is not associated with a value

In [253]:
def scn_to_numpy(resampled_scene, channel):
    # Get the image data and lat/lon arrays
    image_data = resampled_scene[channel].values
    longitudes, latitudes = resampled_scene[channel].attrs['area'].get_lonlats()

    # Create a NumPy array by combining image data with lat/lon information
    combined_array = np.dstack((image_data, latitudes, longitudes))

    # Check that data has been exported sensibly
    plot_combined_data(combined_array)


TypeError: unhashable type: 'list'

In [252]:
def plot_combined_data(combined_array):
    # Assuming combined_array contains image data, latitudes, and longitudes
    image_data = combined_array[:, :, 0]  # Extract image data
    latitudes = combined_array[:, :, 1]  # Extract latitudes
    longitudes = combined_array[:, :, 2]  # Extract longitudes

    # Create a 1x3 grid of subplots
    fig, axs = plt.subplots(3, 1, figsize=(5, 15))

    # Plot longitudes
    axs[0].imshow(longitudes, cmap='viridis', origin='upper')
    axs[0].set_title('Longitude Plot')
    axs[0].set_xlabel('Pixel Column')
    axs[0].set_ylabel('Pixel Row')
    axs[0].set_aspect('equal')  # Equal aspect ratio

    # Plot latitudes
    axs[1].imshow(latitudes, cmap='viridis', origin='upper')
    axs[1].set_title('Latitude Plot')
    axs[1].set_xlabel('Pixel Column')
    axs[1].set_ylabel('Pixel Row')
    axs[1].set_aspect('equal')  # Equal aspect ratio

    # Plot image data
    axs[2].imshow(image_data, cmap='gray', origin='upper')
    axs[2].set_title('Image Data Plot')
    axs[2].set_xlabel('Pixel Column')
    axs[2].set_ylabel('Pixel Row')
    axs[2].set_aspect('equal')  # Equal aspect ratio

    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plots
    plt.show()
